In [ ]:
# End-to-End Berlin Road Crash Analysis Pipeline

This notebook reproduces the complete analysis pipeline linking street-level infrastructure quality (Mapillary images) to road safety outcomes (Berlin crash data).

## Pipeline Overview

1. **Crash and OSM Data Integration**: Aggregate crashes, process OSM road network, match data spatially
2. **Baseline Logistic Regression**: Train OSM-only model to predict crash occurrence
3. **YOLO Feature Extraction**: Extract visual street quality features using YOLOv8
4. **Enhanced Regression**: Train model with YOLO features and compare to baseline
5. **CNN Residual Model**: Train CNN to predict residual risk (unexplained by OSM)
6. **Grad-CAM Visualization**: Generate interpretability visualizations

---

## Setup and Imports

Setting up the environment, paths, and importing necessary modules.


In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

# Set up paths
DATA_RAW = project_root / 'data' / 'raw'
DATA_INTERIM = project_root / 'data' / 'interim'
DATA_PROCESSED = project_root / 'data' / 'processed'
MODELS_DIR = project_root / 'models'
REPORTS_DIR = project_root / 'reports'

print(f"Project root: {project_root}")
print(f"Data directories exist:")
print(f"  - Raw: {DATA_RAW.exists()}")
print(f"  - Interim: {DATA_INTERIM.exists()}")
print(f"  - Processed: {DATA_PROCESSED.exists()}")
print(f"  - Models: {MODELS_DIR.exists()}")


## Section 1: Crash and OSM Data Integration

This section:
- Aggregates crash data from multiple years
- Processes OSM road network
- Matches crashes and images to nearest road segments
- Creates 15m cluster dataset with crash matching

### 1.1 Aggregate Crash Data


In [ ]:
# Check if aggregated crashes already exist
CRASHES_AGGREGATED = DATA_INTERIM / 'crashes_aggregated.csv'

if CRASHES_AGGREGATED.exists():
    print(f"✅ Found existing aggregated crashes: {CRASHES_AGGREGATED}")
    crashes_df = pd.read_csv(CRASHES_AGGREGATED)
    print(f"   Loaded {len(crashes_df):,} crash records")
else:
    print("⚠️  Aggregated crashes not found. Running aggregation...")
    # Import and run aggregation
    from src.features.aggregate_crashes import main as aggregate_crashes
    aggregate_crashes()
    crashes_df = pd.read_csv(CRASHES_AGGREGATED)
    print(f"✅ Created aggregated crashes: {len(crashes_df):,} records")

print(f"\nCrash data summary:")
print(f"  Columns: {list(crashes_df.columns[:10])}...")
print(f"  Year range: {crashes_df['UJAHR'].min()}-{crashes_df['UJAHR'].max()}")
print(f"  Total crashes: {len(crashes_df):,}")


### 1.2 Process OSM Road Network


In [ ]:
# Check if OSM roads already processed
OSM_ROADS = DATA_INTERIM / 'osm_berlin_roads.gpkg'

if OSM_ROADS.exists():
    print(f"✅ Found existing OSM roads: {OSM_ROADS}")
    import geopandas as gpd
    roads_gdf = gpd.read_file(OSM_ROADS)
    print(f"   Loaded {len(roads_gdf):,} road segments")
else:
    print("⚠️  OSM roads not found. Running processing...")
    from src.features.process_osm_efficient import main as process_osm
    process_osm()
    roads_gdf = gpd.read_file(OSM_ROADS)
    print(f"✅ Created OSM roads: {len(roads_gdf):,} segments")

print(f"\nOSM roads summary:")
print(f"  CRS: {roads_gdf.crs}")
print(f"  Columns: {list(roads_gdf.columns[:10])}...")
if 'road_segment_length_m' in roads_gdf.columns:
    print(f"  Total length: {roads_gdf['road_segment_length_m'].sum() / 1000:.1f} km")


### 1.3 Match Crashes and Images to OSM Roads


In [ ]:
# Check if OSM-matched data already exists
CRASHES_WITH_OSM = DATA_PROCESSED / 'crashes_with_osm.csv'
MAPILLARY_WITH_OSM = DATA_PROCESSED / 'mapillary_with_osm.csv'

if CRASHES_WITH_OSM.exists() and MAPILLARY_WITH_OSM.exists():
    print(f"✅ Found existing OSM-matched data")
    crashes_osm = pd.read_csv(CRASHES_WITH_OSM)
    images_osm = pd.read_csv(MAPILLARY_WITH_OSM)
    print(f"   Crashes with OSM: {len(crashes_osm):,} ({crashes_osm['dist_to_road_m'].notna().sum():,} matched)")
    print(f"   Images with OSM: {len(images_osm):,} ({images_osm['dist_to_road_m'].notna().sum():,} matched)")
else:
    print("⚠️  OSM-matched data not found. Running spatial matching...")
    from src.features.snap_to_roads import main as snap_to_roads
    snap_to_roads()
    crashes_osm = pd.read_csv(CRASHES_WITH_OSM)
    images_osm = pd.read_csv(MAPILLARY_WITH_OSM)
    print(f"✅ Created OSM-matched data")

# Calculate match rates
if 'dist_to_road_m' in crashes_osm.columns:
    crash_match_rate = crashes_osm['dist_to_road_m'].notna().sum() / len(crashes_osm) * 100
    print(f"\nCrash match rate: {crash_match_rate:.1f}%")

if 'dist_to_road_m' in images_osm.columns:
    image_match_rate = images_osm['dist_to_road_m'].notna().sum() / len(images_osm) * 100
    print(f"Image match rate: {image_match_rate:.1f}%")


### 1.4 Create 15m Cluster Dataset


In [ ]:
# Check if cluster dataset already exists
CLUSTERS_DATA = DATA_PROCESSED / 'clusters_with_crashes.csv'

if CLUSTERS_DATA.exists():
    print(f"✅ Found existing cluster dataset: {CLUSTERS_DATA}")
    clusters_df = pd.read_csv(CLUSTERS_DATA)
    print(f"   Loaded {len(clusters_df):,} clusters")
    print(f"   Clusters with crashes: {(clusters_df['match_label'] == 1).sum():,}")
else:
    print("⚠️  Cluster dataset not found. Creating clusters...")
    from src.features.create_cluster_dataset import main as create_clusters
    create_clusters()
    clusters_df = pd.read_csv(CLUSTERS_DATA)
    print(f"✅ Created cluster dataset: {len(clusters_df):,} clusters")

print(f"\nCluster dataset summary:")
print(f"  Total clusters: {len(clusters_df):,}")
print(f"  Clusters with crashes: {(clusters_df['match_label'] == 1).sum():,} ({100 * (clusters_df['match_label'] == 1).mean():.1f}%)")
if 'split' in clusters_df.columns:
    print(f"\nSplit distribution:")
    print(clusters_df['split'].value_counts())


## Section 2: Baseline Logistic Regression (OSM-only)

This section:
- Trains a baseline logistic regression using only structured OSM road attributes
- Predicts crash occurrence (binary classification)
- Computes residuals (actual - predicted probability) for CNN target
- Evaluates model performance on train/val/test splits

### 2.1 Load Cluster Data with Splits


In [ ]:
# Check if residuals already computed
CLUSTERS_TRAIN_RES = DATA_PROCESSED / 'clusters_train_with_residuals.csv'
CLUSTERS_VAL_RES = DATA_PROCESSED / 'clusters_val_with_residuals.csv'
CLUSTERS_TEST_RES = DATA_PROCESSED / 'clusters_test_with_residuals.csv'
BASELINE_MODEL = MODELS_DIR / 'baseline_logistic_regression.pkl'

if all([CLUSTERS_TRAIN_RES.exists(), CLUSTERS_VAL_RES.exists(), CLUSTERS_TEST_RES.exists(), BASELINE_MODEL.exists()]):
    print(f"✅ Found existing baseline model and residuals")
    clusters_train = pd.read_csv(CLUSTERS_TRAIN_RES)
    clusters_val = pd.read_csv(CLUSTERS_VAL_RES)
    clusters_test = pd.read_csv(CLUSTERS_TEST_RES)
    print(f"   Train: {len(clusters_train):,} clusters")
    print(f"   Val: {len(clusters_val):,} clusters")
    print(f"   Test: {len(clusters_test):,} clusters")
    
    # Load model metadata
    import joblib
    baseline_model = joblib.load(BASELINE_MODEL)
    with open(MODELS_DIR / 'baseline_logistic_regression_metadata.json', 'r') as f:
        baseline_metadata = json.load(f)
    print(f"   Model trained with {baseline_metadata.get('n_features', 'N/A')} features")
else:
    print("⚠️  Baseline model not found. Training baseline logistic regression...")
    from src.modeling.baseline_logistic_regression import main as train_baseline
    train_baseline()
    
    clusters_train = pd.read_csv(CLUSTERS_TRAIN_RES)
    clusters_val = pd.read_csv(CLUSTERS_VAL_RES)
    clusters_test = pd.read_csv(CLUSTERS_TEST_RES)
    baseline_model = joblib.load(BASELINE_MODEL)
    print(f"✅ Trained baseline model")

# Display residual statistics
if 'residual' in clusters_train.columns:
    print(f"\nResidual statistics:")
    print(f"  Train - Mean: {clusters_train['residual'].mean():.4f}, Std: {clusters_train['residual'].std():.4f}")
    print(f"  Val - Mean: {clusters_val['residual'].mean():.4f}, Std: {clusters_val['residual'].std():.4f}")
    print(f"  Test - Mean: {clusters_test['residual'].mean():.4f}, Std: {clusters_test['residual'].std():.4f}")


### 2.2 Display Baseline Model Performance


In [ ]:
# Load baseline metrics
BASELINE_METRICS = REPORTS_DIR / 'Regression_beforeCNN' / 'baseline_metrics_comparison.csv'

if BASELINE_METRICS.exists():
    baseline_metrics = pd.read_csv(BASELINE_METRICS)
    print("Baseline Logistic Regression Performance:")
    print("=" * 60)
    print(baseline_metrics.to_string(index=False))
    
    # Create visualization
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("whitegrid")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Metrics comparison
    metric_cols = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC_AUC']
    metrics_data = baseline_metrics.set_index('Split')[metric_cols]
    
    metrics_data.plot(kind='bar', ax=axes[0], rot=0)
    axes[0].set_title('Baseline Model Metrics by Split')
    axes[0].set_ylabel('Score')
    axes[0].legend(loc='best')
    axes[0].grid(True, alpha=0.3)
    
    # Residual distribution
    if 'residual' in clusters_test.columns:
        axes[1].hist(clusters_test['residual'], bins=50, alpha=0.7, edgecolor='black')
        axes[1].axvline(0, color='red', linestyle='--', label='Zero residual')
        axes[1].set_xlabel('Residual (Actual - Predicted)')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title('Test Set Residual Distribution')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Baseline metrics not found. Run baseline_logistic_regression.py to generate.")


## Section 3: YOLO Inference and Feature Extraction

This section:
- Loads trained YOLOv8 model
- Extracts visual street quality features using ROI filtering
- Computes per-cluster aggregated metrics
- Winsorizes features to reduce outlier impact

### 3.1 Load YOLO Model and Extract Features


In [ ]:
# Check if YOLO features already extracted
CLUSTERS_TRAIN_YOLO = DATA_PROCESSED / 'clusters_train_with_yolo_roi.csv'
CLUSTERS_VAL_YOLO = DATA_PROCESSED / 'clusters_val_with_yolo_roi.csv'
CLUSTERS_TEST_YOLO = DATA_PROCESSED / 'clusters_test_with_yolo_roi.csv'
YOLO_MODEL_PATH = project_root / 'runs' / 'detect' / 'train' / 'weights' / 'best.pt'

if all([CLUSTERS_TRAIN_YOLO.exists(), CLUSTERS_VAL_YOLO.exists(), CLUSTERS_TEST_YOLO.exists()]):
    print(f"✅ Found existing YOLO features")
    clusters_train_yolo = pd.read_csv(CLUSTERS_TRAIN_YOLO)
    clusters_val_yolo = pd.read_csv(CLUSTERS_VAL_YOLO)
    clusters_test_yolo = pd.read_csv(CLUSTERS_TEST_YOLO)
    print(f"   Train: {len(clusters_train_yolo):,} clusters")
    print(f"   Val: {len(clusters_val_yolo):,} clusters")
    print(f"   Test: {len(clusters_test_yolo):,} clusters")
    
    # Display YOLO feature statistics
    if 'street_quality_roi_winz' in clusters_train_yolo.columns:
        print(f"\nYOLO Feature (street_quality_roi_winz) statistics:")
        print(f"  Train - Mean: {clusters_train_yolo['street_quality_roi_winz'].mean():.4f}, "
              f"Std: {clusters_train_yolo['street_quality_roi_winz'].std():.4f}")
        print(f"  Val - Mean: {clusters_val_yolo['street_quality_roi_winz'].mean():.4f}, "
              f"Std: {clusters_val_yolo['street_quality_roi_winz'].std():.4f}")
        print(f"  Test - Mean: {clusters_test_yolo['street_quality_roi_winz'].mean():.4f}, "
              f"Std: {clusters_test_yolo['street_quality_roi_winz'].std():.4f}")
else:
    if not YOLO_MODEL_PATH.exists():
        print(f"⚠️  YOLO model not found at {YOLO_MODEL_PATH}")
        print("   Please train YOLO model first using train_yolo_cz_no.py")
    else:
        print("⚠️  YOLO features not found. Extracting features...")
        print("   (This may take a while - running YOLO inference on images)")
        from src.modeling.extract_yolo_features import main as extract_yolo
        extract_yolo()
        
        clusters_train_yolo = pd.read_csv(CLUSTERS_TRAIN_YOLO)
        clusters_val_yolo = pd.read_csv(CLUSTERS_VAL_YOLO)
        clusters_test_yolo = pd.read_csv(CLUSTERS_TEST_YOLO)
        print(f"✅ Extracted YOLO features")


### 3.2 Visualize YOLO Feature Distribution


In [ ]:
if all([CLUSTERS_TRAIN_YOLO.exists(), CLUSTERS_VAL_YOLO.exists(), CLUSTERS_TEST_YOLO.exists()]):
    clusters_train_yolo = pd.read_csv(CLUSTERS_TRAIN_YOLO)
    clusters_val_yolo = pd.read_csv(CLUSTERS_VAL_YOLO)
    clusters_test_yolo = pd.read_csv(CLUSTERS_TEST_YOLO)
    
    if 'street_quality_roi_winz' in clusters_train_yolo.columns:
        import matplotlib.pyplot as plt
        import seaborn as sns
        sns.set_style("whitegrid")
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Distribution by split
        splits_data = pd.concat([
            clusters_train_yolo[['street_quality_roi_winz']].assign(Split='Train'),
            clusters_val_yolo[['street_quality_roi_winz']].assign(Split='Val'),
            clusters_test_yolo[['street_quality_roi_winz']].assign(Split='Test')
        ])
        
        splits_data.boxplot(column='street_quality_roi_winz', by='Split', ax=axes[0])
        axes[0].set_title('YOLO Feature Distribution by Split')
        axes[0].set_ylabel('Street Quality ROI (Winsorized)')
        axes[0].set_xlabel('')
        plt.suptitle('')
        
        # Correlation with crash label
        all_yolo = pd.concat([
            clusters_train_yolo[['street_quality_roi_winz', 'match_label']].assign(Split='Train'),
            clusters_val_yolo[['street_quality_roi_winz', 'match_label']].assign(Split='Val'),
            clusters_test_yolo[['street_quality_roi_winz', 'match_label']].assign(Split='Test')
        ])
        
        for split in ['Train', 'Val', 'Test']:
            split_data = all_yolo[all_yolo['Split'] == split]
            axes[1].scatter(
                split_data['street_quality_roi_winz'], 
                split_data['match_label'],
                alpha=0.3, 
                label=split,
                s=20
            )
        
        axes[1].set_xlabel('Street Quality ROI (Winsorized)')
        axes[1].set_ylabel('Crash Occurrence (0/1)')
        axes[1].set_title('YOLO Feature vs Crash Label')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Correlation coefficient
        for split, df in [('Train', clusters_train_yolo), ('Val', clusters_val_yolo), ('Test', clusters_test_yolo)]:
            if 'street_quality_roi_winz' in df.columns and 'match_label' in df.columns:
                corr = df['street_quality_roi_winz'].corr(df['match_label'])
                print(f"{split} correlation (YOLO feature vs crash): {corr:.4f}")


## Section 4: Enhanced Regression with YOLO Features

This section:
- Trains enhanced logistic regression with baseline features + YOLO features
- Compares performance against baseline model
- Generates comparison visualizations and metrics

### 4.1 Train Enhanced Model with YOLO Features


In [ ]:
# Check if enhanced model already trained
YOLO_MODEL = MODELS_DIR / 'logistic_regression_with_yolo.pkl'
YOLO_METRICS = REPORTS_DIR / 'Regression_withYOLO' / 'logistic_regression_with_yolo_metrics.csv'

if YOLO_MODEL.exists() and YOLO_METRICS.exists():
    print(f"✅ Found existing enhanced YOLO model")
    import joblib
    yolo_model = joblib.load(YOLO_MODEL)
    yolo_metrics = pd.read_csv(YOLO_METRICS)
    
    print("\nEnhanced Model (with YOLO) Performance:")
    print("=" * 60)
    print(yolo_metrics.to_string(index=False))
else:
    if not all([CLUSTERS_TRAIN_YOLO.exists(), CLUSTERS_VAL_YOLO.exists(), CLUSTERS_TEST_YOLO.exists()]):
        print("⚠️  YOLO features not found. Run Section 3 first.")
    else:
        print("⚠️  Enhanced model not found. Training enhanced logistic regression...")
        from src.modeling.logistic_regression_with_yolo import main as train_yolo_model
        train_yolo_model()
        
        yolo_model = joblib.load(YOLO_MODEL)
        yolo_metrics = pd.read_csv(YOLO_METRICS)
        print(f"✅ Trained enhanced model")

# Load metadata
if (MODELS_DIR / 'logistic_regression_with_yolo_metadata.json').exists():
    with open(MODELS_DIR / 'logistic_regression_with_yolo_metadata.json', 'r') as f:
        yolo_metadata = json.load(f)
    print(f"\nModel info:")
    print(f"  Features: {yolo_metadata.get('n_features', 'N/A')}")
    print(f"  Includes YOLO feature: street_quality_roi_winz")


### 4.2 Compare Baseline vs Enhanced Model


In [ ]:
# Check if comparison already done
COMPARISON_METRICS = REPORTS_DIR / 'Comparison_Baseline_vs_YOLO' / 'metrics_comparison.csv'

if COMPARISON_METRICS.exists():
    comparison_df = pd.read_csv(COMPARISON_METRICS)
    print("Model Comparison: Baseline vs Enhanced (with YOLO)")
    print("=" * 70)
    print(comparison_df.to_string(index=False))
    
    # Create comparison visualization
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("whitegrid")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Metrics comparison bar chart
    if 'Split' in comparison_df.columns and 'Model' in comparison_df.columns:
        metric_cols = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC_AUC']
        available_metrics = [col for col in metric_cols if col in comparison_df.columns]
        
        if available_metrics:
            comparison_pivot = comparison_df.pivot_table(
                index='Split', 
                columns='Model', 
                values=available_metrics[0]
            )
            comparison_pivot.plot(kind='bar', ax=axes[0], rot=0)
            axes[0].set_title('Model Comparison by Split')
            axes[0].set_ylabel('Score')
            axes[0].legend(title='Model', loc='best')
            axes[0].grid(True, alpha=0.3)
    
    # Delta metrics if available
    DELTA_METRICS = REPORTS_DIR / 'Comparison_Baseline_vs_YOLO' / 'test_metrics_delta.csv'
    if DELTA_METRICS.exists():
        delta_df = pd.read_csv(DELTA_METRICS)
        if len(delta_df) > 0:
            delta_cols = [col for col in delta_df.columns if col.startswith('Δ')]
            if delta_cols:
                delta_df[delta_cols[0]].plot(kind='barh', ax=axes[1])
                axes[1].axvline(0, color='red', linestyle='--', alpha=0.5)
                axes[1].set_title(f'Performance Delta (Enhanced - Baseline)')
                axes[1].set_xlabel('Δ Score')
                axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Comparison metrics not found. Running comparison...")
    from src.modeling.compare_baseline_vs_yolo import main as compare_models
    compare_models()
    comparison_df = pd.read_csv(COMPARISON_METRICS)
    print("✅ Generated comparison metrics")


## Section 5: CNN Residual Model

This section:
- Prepares CNN dataset (one image per cluster with residual label)
- Trains ResNet18 regression model to predict residual risk
- Evaluates model and ranks high-risk images

### 5.1 Prepare CNN Dataset


In [ ]:
# Check if CNN dataset already prepared
CNN_DATASET = DATA_PROCESSED / 'cnn_samples_residual.csv'

if CNN_DATASET.exists():
    print(f"✅ Found existing CNN dataset: {CNN_DATASET}")
    cnn_samples = pd.read_csv(CNN_DATASET)
    print(f"   Loaded {len(cnn_samples):,} samples")
    print(f"   Train: {(cnn_samples['split'] == 'train').sum():,}")
    print(f"   Val: {(cnn_samples['split'] == 'val').sum():,}")
    print(f"   Test: {(cnn_samples['split'] == 'test').sum():,}")
    
    print(f"\nResidual statistics by split:")
    print(cnn_samples.groupby('split')['residual'].agg(['count', 'mean', 'std', 'min', 'max']).round(4))
else:
    if not all([CLUSTERS_TRAIN_RES.exists(), CLUSTERS_VAL_RES.exists(), CLUSTERS_TEST_RES.exists()]):
        print("⚠️  Residual data not found. Run Section 2 first to generate residuals.")
    else:
        print("⚠️  CNN dataset not found. Preparing dataset...")
        from src.modeling.prepare_cnn_dataset import main as prepare_cnn
        prepare_cnn()
        cnn_samples = pd.read_csv(CNN_DATASET)
        print(f"✅ Created CNN dataset: {len(cnn_samples):,} samples")


### 5.2 Train CNN Residual Model


In [ ]:
# Check if CNN model already trained
CNN_MODEL = MODELS_DIR / 'cnn_residual_best.pth'
CNN_PREDICTIONS = DATA_PROCESSED / 'cnn_test_predictions.csv'

if CNN_MODEL.exists():
    print(f"✅ Found existing CNN model: {CNN_MODEL}")
    
    # Check for training log
    TRAINING_LOG = REPORTS_DIR / 'CNN' / 'training_log.csv'
    if TRAINING_LOG.exists():
        training_log = pd.read_csv(TRAINING_LOG)
        print(f"   Training epochs: {len(training_log)}")
        print(f"   Best validation MSE: {training_log['val_mse'].min():.6f}")
        print(f"\nLast 5 epochs:")
        print(training_log.tail().to_string(index=False))
    
    # Check for test predictions
    if CNN_PREDICTIONS.exists():
        cnn_preds = pd.read_csv(CNN_PREDICTIONS)
        print(f"\nTest predictions:")
        print(f"   Samples: {len(cnn_preds):,}")
        print(f"   Mean predicted residual: {cnn_preds['predicted_residual'].mean():.4f}")
        print(f"   Std predicted residual: {cnn_preds['predicted_residual'].std():.4f}")
        
        # Correlation with true residuals
        if 'true_residual' in cnn_preds.columns:
            corr = cnn_preds['predicted_residual'].corr(cnn_preds['true_residual'])
            print(f"   Correlation (predicted vs true): {corr:.4f}")
else:
    if not CNN_DATASET.exists():
        print("⚠️  CNN dataset not found. Run Section 5.1 first.")
    else:
        print("⚠️  CNN model not found.")
        print("   Training CNN can take a long time (20-30 minutes).")
        print("   To train, run: python src/modeling/train_cnn_residual.py")
        print("   Or uncomment the code below to train in this notebook:")
        
        # Option to train (commented out by default)
        # from src.modeling.train_cnn_residual import main as train_cnn
        # train_cnn()


### 5.3 Visualize CNN Results


In [ ]:
if CNN_PREDICTIONS.exists() and CNN_MODEL.exists():
    cnn_preds = pd.read_csv(CNN_PREDICTIONS)
    
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("whitegrid")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot: predicted vs true residuals
    if 'true_residual' in cnn_preds.columns:
        axes[0].scatter(cnn_preds['true_residual'], cnn_preds['predicted_residual'], 
                       alpha=0.5, s=20)
        axes[0].plot([cnn_preds['true_residual'].min(), cnn_preds['true_residual'].max()],
                    [cnn_preds['true_residual'].min(), cnn_preds['true_residual'].max()],
                    'r--', label='Perfect prediction')
        axes[0].set_xlabel('True Residual')
        axes[0].set_ylabel('Predicted Residual')
        axes[0].set_title('CNN Residual Prediction')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        corr = cnn_preds['predicted_residual'].corr(cnn_preds['true_residual'])
        axes[0].text(0.05, 0.95, f'Correlation: {corr:.4f}', 
                    transform=axes[0].transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Distribution of predicted residuals
    axes[1].hist(cnn_preds['predicted_residual'], bins=50, alpha=0.7, edgecolor='black')
    axes[1].axvline(cnn_preds['predicted_residual'].mean(), color='red', 
                   linestyle='--', label=f"Mean: {cnn_preds['predicted_residual'].mean():.4f}")
    axes[1].set_xlabel('Predicted Residual')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Predicted Residual Distribution')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Check for high-risk images ranking
    HIGH_RISK = DATA_PROCESSED / 'cnn_test_top_visual_risks.csv'
    if HIGH_RISK.exists():
        high_risk = pd.read_csv(HIGH_RISK)
        print(f"\nTop 10 highest predicted residual (high-risk) images:")
        print(high_risk.head(10)[['cluster_id', 'predicted_residual', 'true_residual']].to_string(index=False))
else:
    print("⚠️  CNN predictions not found. Train CNN model first (Section 5.2).")


In [ ]:
# Check if Grad-CAM already generated
GRADCAM_DIR = REPORTS_DIR / 'CNN' / 'gradcam'
GRADCAM_REPORT = GRADCAM_DIR / 'gradcam_analysis_report.md'

if GRADCAM_DIR.exists() and GRADCAM_REPORT.exists():
    print(f"✅ Found existing Grad-CAM visualizations: {GRADCAM_DIR}")
    
    # Count generated files
    heatmap_count = len(list(GRADCAM_DIR.glob('*_heatmap.jpg')))
    overlay_count = len(list(GRADCAM_DIR.glob('*_overlay.jpg')))
    grid_count = len(list(GRADCAM_DIR.glob('*_grid.png')))
    
    print(f"   Heatmaps: {heatmap_count}")
    print(f"   Overlays: {overlay_count}")
    print(f"   Grids: {grid_count}")
    
    # Display report summary
    if GRADCAM_REPORT.exists():
        with open(GRADCAM_REPORT, 'r') as f:
            report_content = f.read()
            # Extract key statistics
            print(f"\nGrad-CAM Analysis Report:")
            print("=" * 60)
            lines = report_content.split('\n')[:20]  # First 20 lines
            for line in lines:
                if line.strip():
                    print(line)
            print("\n... (see full report for complete analysis)")
else:
    if not CNN_MODEL.exists():
        print("⚠️  CNN model not found. Train CNN model first (Section 5.2).")
    elif not CNN_DATASET.exists():
        print("⚠️  CNN dataset not found. Run Section 5.1 first.")
    else:
        print("⚠️  Grad-CAM visualizations not found.")
        print("   Generating Grad-CAM can take a while (5-10 minutes).")
        print("   To generate, run:")
        print("   python src/modeling/visualize_gradcam.py --checkpoint models/cnn_residual_best.pth --num_samples 20")
        print("   Or uncomment the code below to generate in this notebook:")
        
        # Option to generate (commented out by default)
        # import argparse
        # from src.modeling.visualize_gradcam import main as visualize_gradcam
        # # Create mock args
        # args = argparse.Namespace(
        #     checkpoint=str(CNN_MODEL),
        #     num_samples=20,
        #     method='gradcam',
        #     colormap='jet',
        #     save_stats=True,
        #     output_dir=str(GRADCAM_DIR)
        # )
        # visualize_gradcam()


### 6.2 Display Sample Grad-CAM Visualizations


In [ ]:
if GRADCAM_DIR.exists():
    from IPython.display import Image, display, Markdown
    import glob
    
    # Find sample grids
    grid_files = sorted(glob.glob(str(GRADCAM_DIR / '*_grid.png')))
    
    if grid_files:
        print("Sample Grad-CAM Grid Visualizations:")
        print("=" * 60)
        
        # Display first few grids
        for i, grid_file in enumerate(grid_files[:2]):
            print(f"\nGrid {i+1}:")
            display(Image(filename=grid_file, width=800))
    else:
        print("⚠️  Grid visualizations not found. Generate Grad-CAM first.")
    
    # Display report if available
    if GRADCAM_REPORT.exists():
        with open(GRADCAM_REPORT, 'r') as f:
            report_md = f.read()
        display(Markdown(report_md))
else:
    print("⚠️  Grad-CAM visualizations not found. Generate them first (Section 6.1).")


## Summary: Key Findings and Model Comparison

This section summarizes the key findings across all pipeline steps and compares model performance.


In [ ]:
print("=" * 80)
print("END-TO-END PIPELINE SUMMARY")
print("=" * 80)

# 1. Data Integration Summary
print("\n1. DATA INTEGRATION:")
print("   " + "-" * 76)
if CLUSTERS_DATA.exists():
    clusters_summary = clusters_df if 'clusters_df' in locals() else pd.read_csv(CLUSTERS_DATA)
    print(f"   Total clusters: {len(clusters_summary):,}")
    print(f"   Clusters with crashes: {(clusters_summary['match_label'] == 1).sum():,} "
          f"({100 * (clusters_summary['match_label'] == 1).mean():.1f}%)")

# 2. Baseline Model Summary
print("\n2. BASELINE MODEL (OSM-only):")
print("   " + "-" * 76)
BASELINE_METRICS = REPORTS_DIR / 'Regression_beforeCNN' / 'baseline_metrics_comparison.csv'
if BASELINE_METRICS.exists():
    baseline_metrics = pd.read_csv(BASELINE_METRICS)
    test_metrics = baseline_metrics[baseline_metrics['Split'] == 'Test']
    if len(test_metrics) > 0:
        row = test_metrics.iloc[0]
        if 'ROC_AUC' in row:
            print(f"   Test ROC AUC: {row['ROC_AUC']:.4f}")
        if 'F1' in row:
            print(f"   Test F1: {row['F1']:.4f}")
        if 'Precision' in row:
            print(f"   Test Precision: {row['Precision']:.4f}")
        if 'Recall' in row:
            print(f"   Test Recall: {row['Recall']:.4f}")

# 3. Enhanced Model Summary
print("\n3. ENHANCED MODEL (OSM + YOLO):")
print("   " + "-" * 76)
YOLO_METRICS = REPORTS_DIR / 'Regression_withYOLO' / 'logistic_regression_with_yolo_metrics.csv'
if YOLO_METRICS.exists():
    yolo_metrics = pd.read_csv(YOLO_METRICS)
    test_yolo = yolo_metrics[yolo_metrics['Split'] == 'Test']
    if len(test_yolo) > 0:
        row = test_yolo.iloc[0]
        if 'ROC_AUC' in row:
            print(f"   Test ROC AUC: {row['ROC_AUC']:.4f}")
        if 'F1' in row:
            print(f"   Test F1: {row['F1']:.4f}")
        if 'Precision' in row:
            print(f"   Test Precision: {row['Precision']:.4f}")
        if 'Recall' in row:
            print(f"   Test Recall: {row['Recall']:.4f}")

# 4. Model Comparison
print("\n4. MODEL COMPARISON:")
print("   " + "-" * 76)
COMPARISON_METRICS = REPORTS_DIR / 'Comparison_Baseline_vs_YOLO' / 'metrics_comparison.csv'
if COMPARISON_METRICS.exists():
    comp_metrics = pd.read_csv(COMPARISON_METRICS)
    test_baseline = comp_metrics[(comp_metrics['Split'] == 'Test') & 
                                  (comp_metrics['Model'] == 'Baseline')]
    test_yolo = comp_metrics[(comp_metrics['Split'] == 'Test') & 
                             (comp_metrics['Model'] == 'Enhanced')]
    
    if len(test_baseline) > 0 and len(test_yolo) > 0:
        bl_row = test_baseline.iloc[0]
        yolo_row = test_yolo.iloc[0]
        
        if 'ROC_AUC' in bl_row and 'ROC_AUC' in yolo_row:
            delta_auc = yolo_row['ROC_AUC'] - bl_row['ROC_AUC']
            print(f"   Δ ROC AUC (Enhanced - Baseline): {delta_auc:+.4f}")
        
        if 'F1' in bl_row and 'F1' in yolo_row:
            delta_f1 = yolo_row['F1'] - bl_row['F1']
            print(f"   Δ F1 (Enhanced - Baseline): {delta_f1:+.4f}")

# 5. CNN Model Summary
print("\n5. CNN RESIDUAL MODEL:")
print("   " + "-" * 76)
CNN_PREDICTIONS = DATA_PROCESSED / 'cnn_test_predictions.csv'
if CNN_PREDICTIONS.exists():
    cnn_preds = pd.read_csv(CNN_PREDICTIONS)
    if 'true_residual' in cnn_preds.columns:
        corr = cnn_preds['predicted_residual'].corr(cnn_preds['true_residual'])
        print(f"   Test correlation (predicted vs true residual): {corr:.4f}")
        print(f"   Test samples: {len(cnn_preds):,}")

# 6. Grad-CAM Summary
print("\n6. GRAD-CAM VISUALIZATION:")
print("   " + "-" * 76)
GRADCAM_DIR = REPORTS_DIR / 'CNN' / 'gradcam'
if GRADCAM_DIR.exists():
    print(f"   Visualizations generated: {GRADCAM_DIR}")
    print(f"   Analysis report available: {(GRADCAM_DIR / 'gradcam_analysis_report.md').exists()}")

print("\n" + "=" * 80)
print("Pipeline complete! All results saved to data/processed/, models/, and reports/")
print("=" * 80)
